# **Probability and Random Processes**
# Assignment 3: Bayesian Matting

In [14]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import matplotlib.pyplot as plt
import cv2
from sklearn.cluster import KMeans
from numba import jit 
from tqdm import tqdm

### Downloading and Unzipping the input images, trimaps and ground truth from the source provided

In [2]:
import shutil
import os

if os.path.exists("data"):
    shutil.rmtree("data")

!wget -q http://www.alphamatting.com/datasets/zip/input_training_lowres.zip
!wget -q http://www.alphamatting.com/datasets/zip/trimap_training_lowres.zip
!wget -q http://www.alphamatting.com/datasets/zip/gt_training_lowres.zip

os.mkdir("data")
os.mkdir("data/Input")
os.mkdir("data/Trimap")
os.mkdir("data/GT")

!unzip -q "input_training_lowres.zip" -d "data/Input"
!unzip -q "trimap_training_lowres.zip" -d "data/Trimap"
!unzip -q "gt_training_lowres.zip" -d "data/GT"
os.remove("input_training_lowres.zip")
os.remove("trimap_training_lowres.zip")
os.remove("gt_training_lowres.zip")

### Loading images and Trimaps in the program

In [3]:
names = sorted(os.listdir("data/Input"))

input_images = []
trimaps = []

for name in names:
    num = name.split('.')[0][-2:]
    img = cv2.imread("data/Input/GT"+num+".png")
    input_images.append(img)
    tri = cv2.imread("data/Trimap/Trimap1/GT"+num+".png", cv2.IMREAD_GRAYSCALE)
    trimaps.append(tri)



### All tunable parameters

In [4]:
### Tuning Parameters
global sigma
sigma = 10              # Default sigma for gaussian weights
global N
N = 25                  # Default window size
global sigma_C
sigma_C = 0.01          # Default variance for the pixel
global maxIter
maxIter = 1000          # Maximum number of iteration in iterative solving function (break condition)
global MinLike
MinLike = 1e-6          # Minimum change in iterative solving necessary for next iteration (break condition)
global clust_var
clust_var = 0.05        # Maximum variance of all the clusters allowed
global MaxClusters
MaxClusters = 6         # Maximum number of clusters in a given window
global minN
minN = 15               # Minimum number of neighbourhood pixels required for solving at given pixel

### Helper Functions for generating gaussian distributions,  extracting window from the image and returning the weighted mean and covariances of clusters

In [5]:
def gaussian_weights(size=N,sigma = sigma):
    half = size//2
    x , y = np.ogrid[-half:half+1,-half:half+1]
    gaussian = np.exp(-(x**2 + y**2)/(2.0*sigma**2))
    gaussian[gaussian<1e-14] = 0
    return gaussian

def window(image,x,y,N):
    ## Same function for both alpha and the foreground/background. The only change
    ## is the number of channels that are read and  returned in the window
    halfN = N//2
    if len(image.shape)==3:
        h,w,c = image.shape
        xmin = max(0, x - halfN); xmax = min(w, x + (halfN+1))
        ymin = max(0, y - halfN); ymax = min(h, y + (halfN+1))
        pxmin = halfN - (x-xmin); pxmax = halfN + (xmax-x)
        pymin = halfN - (y-ymin); pymax = halfN + (ymax-y)    
        window = np.zeros((N,N,c))
        window[pymin:pymax, pxmin:pxmax] = image[max(0,y - halfN):min(h, y+halfN+1) ,max(0,x - halfN):min(w, x+halfN+1),:]
        return window
    elif len(image.shape)==2:
        h,w = image.shape
        xmin = max(0, x - halfN); xmax = min(w, x + (halfN+1))
        ymin = max(0, y - halfN); ymax = min(h, y + (halfN+1))
        pxmin = halfN - (x-xmin); pxmax = halfN + (xmax-x)
        pymin = halfN - (y-ymin); pymax = halfN + (ymax-y)
        window = np.zeros((N,N))
        window[pymin:pymax, pxmin:pxmax] = image[max(0,y - halfN):min(h, y+halfN+1) ,max(0,x - halfN):min(w, x+halfN+1)]
        return window

## Numba jit is a tool for accelerating the performance of numpy functions
@jit(nopython=True, cache=True)
def weighted_mean_cov(pixels,weights):
    W = np.sum(weights)
    mean = (1/W) * np.dot(weights.T,pixels)
    cov = (1/W) * np.dot(weights*(pixels-mean).T,(pixels-mean))
    return mean, cov

### The iterative function that solves iteratively for alpha, foreground and background at a pixel value, provided the pixels, weights and other parameters

In [10]:
@jit(nopython=True, cache=True)
def solve_iterative(front_mu, front_cov, back_mu, back_cov, pixel, sigma_C, alpha_init, MaxIter, MinLike):
    I = np.eye(3)
    FMax = np.zeros(3)
    BMax = np.zeros(3)
    Max_alpha = 0
    MaxLike = - np.inf
    sigma_factor = 1/sigma_C**2

    for i in range(front_mu.shape[0]):
        mu_Fi = front_mu[i]
        inverse_f = np.linalg.inv(front_cov[i])
        for j in range(back_mu.shape[0]):
            mu_Bj = back_mu[j]
            inverse_b = np.linalg.inv(back_cov[j])

            # Initialising the parameters
            alpha = alpha_init
            iter = 1
            prev_like = - np.inf
            
            while True:
                ## SETTING UP THE A AND B MATRICES
                A11 = inverse_f + I*alpha**2 * sigma_factor
                A12 = I*alpha*(1-alpha) * sigma_factor
                A22 = inverse_b+I*(1-alpha)**2 * sigma_factor
                A = np.vstack((np.hstack((A11, A12)), np.hstack((A12, A22))))

                b1 = np.dot(inverse_f, mu_Fi) + pixel*(alpha) * sigma_factor
                b2 = np.dot(inverse_b, mu_Bj) + pixel*(1-alpha) * sigma_factor
                b = np.atleast_2d(np.concatenate((b1, b2))).T    # atleast_2d

                # SOLVING FOR AX = B
                X = np.linalg.solve(A, b)
                F = np.maximum(0, np.minimum(1, X[0:3]))
                B = np.maximum(0, np.minimum(1, X[3:6]))

                alpha = np.maximum(0, np.minimum(1, np.dot((np.atleast_2d(pixel).T-B).T, (F-B))/np.sum((F-B)**2)))[0,0]

                # CALCULATING LOG LIKELIHOOD AND ADDING THEM AND COMPARING WITH THE LAST ITERATION
                L_pixel = - np.sum((np.atleast_2d(pixel).T -alpha*F-(1-alpha)*B)**2) * sigma_factor
                L_F = (- ((F- np.atleast_2d(mu_Fi).T).T @ np.dot(inverse_f , (F-np.atleast_2d(mu_Fi).T)))/2)[0,0]
                L_B = (- ((B- np.atleast_2d(mu_Bj).T).T @ np.dot(inverse_b , (B-np.atleast_2d(mu_Bj).T)))/2)[0,0]
                this_like= (L_pixel + L_F + L_B)
                
                ## Checking if the log likelihood is increasing or not
                if this_like> MaxLike:
                    Max_alpha = alpha
                    MaxLike = this_like
                    FMax = F.flatten()
                    BMax = B.flatten()

                ## Breaking Condition, either reaching maximum iterations, or converging
                if iter >= MaxIter or abs(this_like - prev_like) <= MinLike:
                    break

                prev_like = this_like
                iter += 1
    return FMax, BMax, Max_alpha

### The clustering function (implemented using Scikit-learn KMeans Clustering)

In [7]:
## CLUSTERING USING KMEANS CLUSTERING
def Kcluster(pixels,weights, MinVar = clust_var, MaxClusters = MaxClusters):
    n_clusters = 1
    all_means = []
    all_covs = []
    mean, cov = weighted_mean_cov(pixels,weights)
    cov = cov + 1e-5*np.eye(3)
    all_means.append(mean)
    all_covs.append(cov)
    values, vectors = np.linalg.eig(cov)
    max_value = np.max(values)
    
    ## Condition for forming one more cluster (if the variance of individual clusters is higher than MinVar)
    while max_value>MinVar and n_clusters<MaxClusters:
        n_clusters += 1
        km = KMeans(n_clusters = n_clusters).fit(pixels)
        all_max = []
        for i in range(n_clusters):
            idx = np.where(km.labels_ == i)[0]
            cluster = pixels[idx]
            weight = weights[idx]
            mean, cov = weighted_mean_cov(cluster,weight)
            cov = cov + 1e-5*np.eye(3)
            all_means.append(mean)
            all_covs.append(cov)
            values, vectors = np.linalg.eig(cov)
            all_max.append(np.max(values))
        all_max = np.array(all_max)
        max_value = np.max(all_max)
    all_means = np.array(all_means)
    all_covs = np.array(all_covs)
    return all_means, all_covs
     

### Bayesian matting function which takes an image and its trimap as the input and returns the alpha matte

In [34]:
def Bayesian_matting(image,trimap,N=N, minN = minN):
    h,w,c = image.shape
    image = image/255
    alpha = np.zeros((h,w))
    
    ## Separating the trimap into foreground, background and unknown regions
    fg_mask = trimap == 255
    bg_mask = trimap == 0
    unknown_mask = trimap == 128

    ## Getting the Guassian weights
    weights = gaussian_weights(size = N)
    g_weights = weights/np.max(weights)

    ## Extracting the foregound based on the regions from trimap
    foreground = image * np.repeat(fg_mask[:,:,np.newaxis],3, axis = 2)
    background = image * np.repeat(bg_mask[:,:,np.newaxis],3, axis = 2)

    ## Initialising the alpha matte
    alpha[fg_mask] = 1
    alpha[unknown_mask] = np.nan
    n_unknown_pixels = np.sum(unknown_mask)

    n = 1
    unkreg = unknown_mask 
    kernel = np.ones((3, 3))

    ## Initiaiting the while loop on all the unknown pixels
    while n<n_unknown_pixels:
        leftout = 0
        unkreg = np.array(unkreg.astype(np.uint8))
        unkreg = cv2.erode(unkreg, kernel, iterations=1)
        unkpixels = np.logical_and(np.logical_not(unkreg), unknown_mask)
        
        Y, X = np.nonzero(unkpixels)
        for i in range(Y.shape[0]):
            if n % 1000 == 0:
                print(n, n_unknown_pixels)
                
            y, x = Y[i], X[i]
            p = image[y, x]
            
            ## Getting smaller windows to solve locally
            a_window = window(alpha,x,y,N)
            foreground_window = window(foreground,x,y,N)
            background_window = window(background,x,y,N)
            front = np.reshape(foreground_window, (N*N, 3))
            back = np.reshape(background_window, (N*N, 3))
            
            ## Getting the correct weights by multiplying alpha and gaussian weights
            front_weights = (a_window**2 * g_weights).flatten()
            back_weights = ((1-a_window)**2 * g_weights).flatten()
            
            ## Extracting valid information of foreground and background
            indices = np.nan_to_num(front_weights) > 0
            front = front[indices,:]
            front_weights = front_weights[indices]
            
            indices = np.nan_to_num(back_weights) > 0
            back_weights = back_weights[indices]
            back = back[indices,:]


            ## Checking if the neighbourhood has sufficient information to solve, if not, skip the iteration
            if len(front_weights) < minN or len(back_weights) < minN:
                leftout += 1
                continue

            ## Clustering the colours to get weighted mean and variances
            front_mu, front_cov = Kcluster(front, front_weights)
            back_mu, back_cov = Kcluster(back, back_weights)
            
            ## Solving for F, B and alpha
            alpha_init = np.nanmean(alpha.flatten())
            F, B, Alpha = solve_iterative(front_mu, front_cov, back_mu, back_cov, p, sigma_C, alpha_init, maxIter, MinLike)

            foreground[y, x] = F.flatten()
            alpha[y, x] = Alpha
            background[y, x] = B.flatten()
            n += 1
            unknown_mask[y, x] = 0
        
        ## Keeping track of the unknown pixels and therefore preventing the infinite loop by increasing the window size
        remain = n_unknown_pixels - n
        if remain == leftout-1:
            print("Infinite Loop")
            N = N + 2
            ## Additional condition reqiured in some images to prevent excessive smudging
            # if N>60:
            #     return alpha
            weights = gaussian_weights(size = N)
            g_weights = weights/np.max(weights)

    return alpha

## Main function: Reads the images and trimaps one at a time, and saves two images
1. Alpha Matte in original dimensions
2. Plt figure with the associated error score

In [ ]:
# Change the ranges in the range of the for loop to run matting for custom images
# All alphas are submitted in the zip file along with the report
for img_num in range(11,12):
    GT = cv2.imread("/content/data/GT/GT{:02d}.png".format(img_num),cv2.IMREAD_GRAYSCALE).flatten()
    alpha = 255 * Bayesian_matting(input_images[img_num-1],trimaps[img_num-1])
    cv2.imwrite("alpha_{}.png".format(img_num),alpha)
    alpha = cv2.imread("alpha_{}.png".format(img_num), cv2.IMREAD_GRAYSCALE)
    total = len(GT)

    plt.imshow(alpha, cmap = 'gray')
    plt.title("Alpha matte Score: {:.2f}".format(np.sum(np.absolute(alpha.flatten() - GT))/total))
    plt.axis('off')
    plt.savefig("Output_{}.png".format(img_num))

1000 61930
2000 61930
3000 61930
4000 61930
5000 61930
6000 61930
7000 61930
8000 61930
9000 61930
10000 61930
11000 61930
12000 61930
13000 61930
14000 61930
15000 61930
16000 61930
17000 61930
18000 61930
19000 61930
20000 61930
21000 61930
22000 61930
23000 61930
24000 61930
25000 61930
26000 61930
27000 61930
28000 61930
29000 61930
30000 61930
31000 61930
32000 61930
33000 61930
34000 61930
35000 61930
36000 61930
37000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
38000 61930
39000 61930
40000 61930
